In [ ]:
from google.colab import files
uploaded = files.upload()

Saving train_dataset.csv to train_dataset.csv


In [ ]:
!pip install -U transformers sentencepiece

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 75.9 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.50.0
    Uninstalling transformers-4.50.0:
      Successfully uninstalled transformers-4.50.0


In [ ]:
from transformers import MarianMTModel, MarianTokenizer
from tqdm import tqdm
import torch
import pandas as pd

In [ ]:
df = pd.read_csv(next(iter(uploaded)))
en_de_model_name = 'Helsinki-NLP/opus-mt-en-de'
de_en_model_name = 'Helsinki-NLP/opus-mt-de-en'

en_de_tokenizer = MarianTokenizer.from_pretrained(en_de_model_name)
de_en_tokenizer = MarianTokenizer.from_pretrained(de_en_model_name)

en_de_model = MarianMTModel.from_pretrained(en_de_model_name).to("cuda" if torch.cuda.is_available() else "cpu")
de_en_model = MarianMTModel.from_pretrained(de_en_model_name).to("cuda" if torch.cuda.is_available() else "cpu")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/768k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/797k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.27M [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/298M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/298M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/298M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [ ]:
def translate(texts,tokenizer,model,device):
  inputs = tokenizer(texts, return_tensors="pt", padding=True, truncation= True, max_length=128).to(device)
  outputs = model.generate(**inputs, max_length = 128)
  return [tokenizer.decode(t, skip_special_tokens=True) for t in outputs]

In [ ]:
def back_translate(texts,batch_size=8):
  device = "cuda" if torch.cuda.is_available() else "cpu"
  back_translations = []
  for i in tqdm(range(0,len(texts),batch_size)):
    batch = texts[i:i+batch_size]
    de = translate(batch,tokenizer=en_de_tokenizer,model=en_de_model,device=device)
    en_texts = translate(de,tokenizer=de_en_tokenizer,model=de_en_model,device=device)
    back_translations.extend(en_texts)
  return back_translations

In [ ]:
original_texts = df['text'].tolist()
bt_texts = back_translate(original_texts)

100%|██████████| 123/123 [04:39<00:00,  2.28s/it]


In [ ]:
augmented_rows = []
for orig, bt, label in zip(original_texts, bt_texts, df['label']):
    if orig.strip().lower() != bt.strip().lower():
        augmented_rows.append((label, bt))

In [ ]:
aug_df = pd.DataFrame(augmented_rows, columns=['label', 'text'])

In [ ]:
final_df = pd.concat([df, aug_df], ignore_index=True)
print(final_df.shape)

(1960, 2)


In [ ]:
final_df.to_csv('augmented_train_data.csv', index=False)
files.download('augmented_train_data.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>